In [1]:
from analyze_csv import *
from mutations import get_all_moves
from evaluation_indices import *
from batch_tasks import *
from data_generation import *
from individual import Individual
import numpy as np
import pandas as pd
import os
import traceback
import pickle
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from scipy.stats import entropy, skew, kurtosis
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import f1_micro
from time import time
from random import shuffle

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
all_moves = np.array(get_all_moves())
all_moves

array(['unguided_merge_gene_move', 'unguided_remove_and_reclassify_move',
       'unguided_split_gene_move', 'expand_cluster_move',
       'split_farthest_move', 'unguided_eliminate_move',
       'knn_reclassification_move', 'one_nth_change_move',
       'centroid_k_means_move', 'centroid_hill_climbing_move',
       'prototype_k_means_move', 'prototype_hill_climbing_move',
       'tree_hill_climbing_move',
       'guided_remove_and_reclassify_move("density_based_validity_separation")',
       'guided_remove_and_reclassify_move("density_based_sparseness_separation")',
       'guided_remove_and_reclassify_move("mean_centroid_distance_separation")',
       'guided_remove_and_reclassify_move("diameter_separation")',
       'guided_histogram_split_gene_move("density_based_validity_separation")',
       'guided_histogram_split_gene_move("density_based_sparseness_separation")',
       'guided_histogram_split_gene_move("mean_centroid_distance_separation")',
       'guided_histogram_split_gene_

In [3]:
index_minimized = {}
for name, func in indices:
    index_minimized[name] = eval(func).is_minimized
index_minimized

{'silhouette': False,
 'calinski_harabaz': False,
 'davies_bouldin': True,
 'dvcb': False,
 'dunn': False,
 'generalized_dunn_41': False,
 'generalized_dunn_43': False,
 'generalized_dunn_51': False,
 'generalized_dunn_53': False,
 'generalized_dunn_13': False}

In [4]:
if False:
    mutation_usage = {}
    def parse_folder(path):
        for idx, filename in enumerate(os.listdir(path)):
            index_name = filename[:filename.find('-')]
            try:
                data, time = read_results(path + '/' + filename)
            except:
                traceback.print_exc()
                continue
            n_clusters = data.iloc[-1]['n_clusters']
            unique_numbers = np.unique(np.bincount(data.index[1:]))
            mutation_names = np.array([s[:s.find(':')] if s is not None and s != 'None' else s for s in data['detail']])
            mut_indices = np.argwhere(mutation_names[:, None] == all_moves[None, :])[:, 1]
            assert len(mut_indices) == len(data) - 1
            data['mutation_index'] = np.hstack([[-1], mut_indices])
            index_series = [t['index'].rename(mutation_names[i] + '_index') for i, t in data[['mutation_index', 'index']].groupby('mutation_index')]
            table = pd.concat(index_series, axis=1).values
            if not index_minimized[index_name]:
                max_indices = np.nanargmax(table, axis=1)
                max_values = np.nanmax(table, axis=1)
                change_steps = np.argwhere(max_values[1:] > max_values[:-1]).flatten() + 1
            else:
                max_indices = np.nanargmin(table, axis=1)
                max_values = np.nanmin(table, axis=1)
                change_steps = np.argwhere(max_values[1:] < max_values[:-1]).flatten() + 1
            c_usage = np.unique(max_indices[change_steps]) - 1
            diff = 0 if filename not in mutation_usage else len(np.setdiff1d(c_usage, mutation_usage[filename])) + len(np.setdiff1d(mutation_usage[filename], c_usage))
            mutation_usage[filename] = c_usage if filename not in mutation_usage else np.unique(np.hstack([c_usage, mutation_usage[filename]]))
            print(idx, filename, n_clusters, len(unique_numbers), len(c_usage), len(change_steps), diff)
            
    parse_folder('/tmp/results-1010')
    parse_folder('/tmp/results2')
    mutation_usage = list(mutation_usage.items())
    with open('mutation_usage.dat', 'wb') as f:
        pickle.dump(mutation_usage, f)
else:
    with open('mutation_usage.dat', 'rb') as f:
        mutation_usage = pickle.load(f)

In [5]:
np.bincount(np.hstack([a[1] for  a in mutation_usage]))

array([462, 519, 254, 427, 246, 549, 265, 336, 461, 224, 469, 399, 388,
       394, 397, 356, 242, 144, 172, 232, 241, 208, 225, 360, 354, 350,
       322, 540, 529, 147,  60])

In [6]:
generated_prefix = 'scratch'
real_prefix = 'datasets'
tasks_list = init_batch(real_prefix)
datasDict = {a: {'dataname': a, 'dataset': eval(b)} for a, b in datas}
datasDict.keys()

Excluding column number 2 because it is a single value
Excluding column number 23 because it is a single value


dict_keys(['generated_2dim_10cl', 'generated_2dim_30cl', 'generated_10dim_10cl', 'generated_10dim_30cl', 'immunotherapy', 'user_knowledge', 'sales_transactions', 'Concrete_Data', 'HappinessRank_2015', 'ICU', 'ParkinsonSpeechDataset', 'analcatdata_lawsuit', 'analcatdata_vineyard', 'analcatdata_wildcat', 'arsenic_female_bladder', 'arsenic_male_bladder', 'arsenic_male_lung', 'autoUniv_au7_1100', 'autoUniv_au7_500', 'autoUniv_au7_700', 'badges2', 'banknote_authentication', 'boston_corrected', 'breast_tissue', 'chscase_census2', 'chscase_census6', 'collins', 'dataset_185_yeast', 'dataset_187_abalone', 'dataset_18_mfeat_morphological', 'dataset_2181_auto_price', 'dataset_36_segment', 'dataset_39_ecoli', 'dataset_41_glass', 'dataset_43_haberman', 'dataset_53_heart_statlog', 'dataset_54_vehicle', 'dataset_8_liver_disorders', 'debutanizer', 'disclosure_x_bias', 'fri_c0_1000_25', 'fri_c0_100_25', 'fri_c0_250_10', 'fri_c1_1000_5', 'fri_c1_100_10', 'fri_c1_100_5', 'fri_c1_250_10', 'fri_c1_250_5', 

In [7]:
for k, v in datasDict.items():
    dataset = v['dataset']
    v['ma1'] = np.log2(len(dataset))
    v['ma2'] = np.log2(dataset.shape[1])
    meta_attrs = []
    meta_entropy = []
    cont_attrs = []
    for i, s in enumerate(dataset.T):
        uniq = np.unique(s)
        uniq.sort()
        if len(uniq) < len(s) * 0.3:
            meta_attrs.append(i)
            ints = np.searchsorted(uniq, s)
            meta_entropy.append(entropy(ints))
        else:
            cont_attrs.append(i)
    v['ma3'] = len(meta_attrs)
    v['ma5'] = sum(meta_entropy) / len(meta_entropy) if len(meta_entropy) > 0 else 0
    # TODO ma6
    if len(cont_attrs) > 0:
        corr = np.abs(np.corrcoef(dataset[:, cont_attrs], rowvar=False))
        v['ma7'] = corr[np.arange(0, len(corr))[None, :] <= np.arange(0, len(corr))[:, None]].mean() if isinstance(corr, np.ndarray) else corr
        v['ma8'] = np.mean(skew(dataset[:, cont_attrs]))
        v['ma9'] = np.mean(kurtosis(dataset[:, cont_attrs]))
    else:
        v['ma7'], v['ma8'], v['ma9'] = 0, 0, 0
    dists = pdist(dataset)
    v['md1'] = dists.mean()
    v['md2'] = dists.var()
    v['md3'] = dists.std()
    v['md4'] = skew(dists)
    v['md5'] = kurtosis(dists)
    hist = np.histogram(dists, bins=9, range=(0, dists.max()))[0]
    assert len(hist) == 9
    for i, h in enumerate(hist):
        v['md' + str(i + 6)] = h
    z = np.abs(((dists - v['md1']) / v['md3']))
    v['md16'] = (z < 1).sum()
    v['md17'] = ((1 <= z) & (z < 2)).sum()
    v['md18'] = ((2 <= z) & (z < 3)).sum()
    v['md19'] = (3 <= z).sum()

In [8]:
from sklearn.cluster import KMeans, AffinityPropagation, AgglomerativeClustering, MeanShift
state_of_the_art = ['KMeans(n_clusters=n_clusters)', 'AffinityPropagation()', 'AgglomerativeClustering(n_clusters=n_clusters)', 'MeanShift()']

In [9]:
for k, v in datasDict.items():
    start = time()
    dataset = v['dataset']
    n_clusters = int(np.cbrt(len(dataset)))
    values = []
    fnum = 1
    for algo in state_of_the_art:
        try:
            cls = eval(algo).fit_predict(dataset)
        except:
            traceback.print_exc()
            cls = None
        indiv = Individual({'data': dataset, 'labels': cls})
        for _, idxstr in indices:
            try:
                idx = eval(idxstr)(indiv)
            except:
                if cls is not None:
                    traceback.print_exc()
                idx = np.nan
            v['ml' + str(fnum)] = idx
            values.append(idx)
            fnum += 1
    print(k, 'took', time() - start, 'seconds,', values)

/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)


generated_2dim_10cl took 21.115612506866455 seconds, [0.709873026556441, 20689.835549467483, 3.513413420143543, 0.0438323486456366, 0.037868448519518635, 0.5359873194599644, 0.12212828007784197, 4.6915405849502685, 1.068998764994415, 0.008628578175253945, nan, 979.3979928620977, 1.8281120866197924, 0.12884633564087503, 0.0006471633356246015, 0.0006471633356246015, 0.00014777261905393274, 0.0009707450034369022, 0.0002216589285808991, 0.00014777261905393274, 0.7080526112138442, 20532.027486792307, 3.5235245084209232, 0.04490259890482793, 0.08152223388414177, 0.5133660881799031, 0.1169738818874883, 4.523546782766988, 1.0307202584337505, 0.018575383877374514, 0.3560016935234579, 4258.397449231275, 9.372024621639703, 0.273267171822439, 0.5122049534716551, 1.3299988859013268, 0.09272459668541633, 34.57504419086646, 2.4104960251947087, 0.03570980264298902]


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)
Traceback (most recent call last):
  File "<ipython-input-9-65b6e005b459>", line 16, in <module>
    idx = eval(idxstr)(indiv)
  File "/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/evaluation_indices.py", line 11, in __call__
    return idx(*args, **kwargs)
  File "/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/evaluation_indices.py", line 20, in silhouette_index
    return silhouette_score(indiv["data"], indi

generated_2dim_30cl took 17.767136335372925 seconds, [nan, 2252.2723227073557, 5.109952215573632, 0.11745088069411688, 0.056676282684684026, 0.7220312012425686, 0.10949359163988173, 5.201531086265349, 0.7887946112045534, 0.008594766737592094, nan, 337.6163226994954, 1.7506957972029402, 0.008541400165164335, 0.005414266524697948, 0.005414266524697948, 0.0010973287492174442, 0.008121399787046922, 0.0016459931238261664, 0.0010973287492174442, 0.6424070111975099, 2219.320042355842, 5.459408548171235, 0.15134879749062186, 0.19607973875088028, 0.6763873234364891, 0.11773683768501302, 7.448785157122927, 1.2965890675463905, 0.03413104825404056, nan, nan, 0.0, -0.5090521831735889, nan, nan, nan, nan, nan, nan]


/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)


...

/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)


wholesale_customers took 3.7197253704071045 seconds, [0.2705970697501732, 146.94539157346236, 6.490194764379114, -0.35141072079788016, 0.02662240018504585, 0.15717970428850933, 0.08390881354538827, 1.8499447687193122, 0.9875745177813208, 0.014212102150016764, 0.03444985367353424, 39.73559036534157, 8.341041557673808, -0.2506083175412096, 0.019629728443646046, 0.03135662603022589, 0.011809818833411529, 0.2996414761481557, 0.11285370896966457, 0.007393127578364496, 0.35639253124408976, 141.68062854437574, 4.948000246649376, -0.3457744654920107, 0.05309368405841414, 0.2432692246591342, 0.09666134616071585, 1.6510116749834123, 0.6560180855369974, 0.021096408643177512, 0.4314361953309489, 39.15060518860407, 3.5027402974655004, -0.27292924126133233, 0.08907906159204873, 0.45924092999783894, 0.05925816001852593, 0.6888613949967584, 0.0888872400277889, 0.011494317995886365]


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)


wine_quality_red took 27.74838638305664 seconds, [0.09590398099814094, 282.93415729465124, 9.181621385899755, -0.3213760375239394, 0.021183776961569287, 0.1103006608446523, 0.05290043823305789, 1.8044460862489125, 0.865416290342395, 0.010159785772060517, -0.47339585891592423, 15.660159420753539, 53.31538924965128, -0.3846203953608359, 0.0005130544362764781, 0.0314031397233411, 0.006360774565015212, 0.35832496367265504, 0.07257950431131377, 0.00010392029706220827, nan, 239.09586388495245, 11.696447559092611, -0.30321581360762717, 0.041189813324156145, 0.10718898454865108, 0.041480748510993586, 1.7128944587097346, 0.6628679669538593, 0.01593992419005144, 0.3609326542053717, 53.90640877208247, 7.432429951391653, -0.307638772796813, 0.05558641654022965, 0.43824706636361105, 0.037768663480979195, 0.7448319731127678, 0.06419052242788886, 0.004790504766732629]
yagoSchema_ttl took 0.8185553550720215 seconds, [0.44042967166738795, 200.00355284870716, 3.9296301307838064, -0.44529453782511913, 0.

/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)


In [10]:
datanames = list(datasDict)
shuffle(datanames)
cv10 = [datanames[len(datanames)*i//10:len(datanames)*(i+1)//10] for i in range(10)]
dataset_to_cv = {}
for i, c in enumerate(cv10):
    for v in c:
        dataset_to_cv[v] = i
with open('cv10.dat', 'wb') as f:
    pickle.dump(cv10, f)

In [11]:
meta_knowledge = [{} for i in range(10)]
def get_meta_list(dataset, name):
    meta_knowledge_list = meta_knowledge[dataset_to_cv[dataset]]
    if name not in meta_knowledge_list:
        meta_knowledge_list[name] = []
    return meta_knowledge_list[name]

In [12]:
for fname, mutations in mutation_usage:
    index, data = fname.split('-')[:2]
    get_meta_list(data, 'index').append(index)
    metadata = datasDict[data]
    for k, v in metadata.items():
        if k == 'dataset': continue
        get_meta_list(data, k).append(v)
    arr = np.zeros(len(all_moves), dtype='bool')
    arr[mutations] = True
    for i, v in enumerate(arr):
        get_meta_list(data, 'rm' + str(i)).append(v)
for d in meta_knowledge:
    assert len(set(map(len, d.values()))) == 1

In [13]:
metaframes = [pd.DataFrame(d) for d in meta_knowledge]
metaframes[0].head()

,index,dataname,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,calinski_harabaz,arsenic_male_bladder,9.126704,2.321928,4,5.578625,1.000000,9.600139,104.073434,0.060484,...,False,True,False,True,True,False,True,False,False,False
1,calinski_harabaz,breast_tissue,6.714246,3.321928,1,3.953497,0.293765,0.015479,-1.196633,0.225433,...,False,False,False,False,False,True,False,False,False,False
2,calinski_harabaz,fri_c3_1000_10,9.965784,3.459432,1,6.086775,0.292593,0.228655,-0.705256,0.140777,...,False,False,False,True,True,False,True,False,False,False
3,calinski_harabaz,fri_c4_500_25,8.965784,4.700440,0,0.000000,0.126482,0.107719,-0.941671,0.319531,...,False,False,False,False,True,True,True,False,False,False
4,calinski_harabaz,heart_h,8.194757,3.807355,13,4.836237,1.000000,-0.455030,2.491825,0.157464,...,False,False,False,False,False,False,False,False,False,False


In [14]:
train_columns = [c for c in metaframes[0] if not c.startswith('rm') and c != 'dataname']
test_columns = [c for c in metaframes[0] if c.startswith('rm')]

In [15]:
indices_list = np.array(list(index_minimized))
for d in metaframes:
    d['index'] = np.argwhere(d['index'].values[:, None] == indices_list[None, :])[:, 1].flatten()
    d[test_columns] = d[test_columns].astype('int')
metaframes[0].head()

,index,dataname,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,arsenic_male_bladder,9.126704,2.321928,4,5.578625,1.000000,9.600139,104.073434,0.060484,...,0,1,0,1,1,0,1,0,0,0
1,1,breast_tissue,6.714246,3.321928,1,3.953497,0.293765,0.015479,-1.196633,0.225433,...,0,0,0,0,0,1,0,0,0,0
2,1,fri_c3_1000_10,9.965784,3.459432,1,6.086775,0.292593,0.228655,-0.705256,0.140777,...,0,0,0,1,1,0,1,0,0,0
3,1,fri_c4_500_25,8.965784,4.700440,0,0.000000,0.126482,0.107719,-0.941671,0.319531,...,0,0,0,0,1,1,1,0,0,0
4,1,heart_h,8.194757,3.807355,13,4.836237,1.000000,-0.455030,2.491825,0.157464,...,0,0,0,0,0,0,0,0,0,0


In [20]:
indices_list

array(['silhouette', 'calinski_harabaz', 'davies_bouldin', 'dvcb', 'dunn',
       'generalized_dunn_41', 'generalized_dunn_43',
       'generalized_dunn_51', 'generalized_dunn_53',
       'generalized_dunn_13'], dtype='<U19')

In [16]:
for i, d in enumerate(metaframes):
    d.to_csv('metaframe_{:02d}.csv'.format(i))

In [2]:
metaframes = [pd.read_csv('metaframe_{:02d}.csv'.format(i), index_col=0) for i in range(10)]
metaframes[0].head()

,index,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,md2,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,7.643856,4.321928,17,3.287976,0.522758,0.023865,0.174592,0.142564,0.002888,...,0,0,0,0,0,1,0,0,0,0
1,1,10.396605,2.321928,1,6.413459,0.655107,-0.122684,0.134221,0.066743,0.000965,...,0,0,0,0,0,1,1,1,0,0
2,1,8.413628,2.807355,7,5.592515,0.000000,0.000000,0.000000,0.081610,0.002688,...,0,0,0,0,1,0,0,0,1,0
3,1,9.370687,2.000000,0,0.000000,0.439122,0.349144,0.757310,0.030512,0.000141,...,0,0,0,0,1,1,0,1,0,0
4,1,7.965784,3.459432,0,0.000000,0.263500,-0.038811,-1.128003,0.290846,0.003762,...,0,0,0,0,1,1,1,0,0,0


In [17]:
metaframe_train = pd.concat(metaframes[:-1])
metaframe_train.head()

,index,dataname,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,arsenic_male_bladder,9.126704,2.321928,4,5.578625,1.000000,9.600139,104.073434,0.060484,...,0,1,0,1,1,0,1,0,0,0
1,1,breast_tissue,6.714246,3.321928,1,3.953497,0.293765,0.015479,-1.196633,0.225433,...,0,0,0,0,0,1,0,0,0,0
2,1,fri_c3_1000_10,9.965784,3.459432,1,6.086775,0.292593,0.228655,-0.705256,0.140777,...,0,0,0,1,1,0,1,0,0,0
3,1,fri_c4_500_25,8.965784,4.700440,0,0.000000,0.126482,0.107719,-0.941671,0.319531,...,0,0,0,0,1,1,1,0,0,0
4,1,heart_h,8.194757,3.807355,13,4.836237,1.000000,-0.455030,2.491825,0.157464,...,0,0,0,0,0,0,0,0,0,0


In [7]:
cls = AutoSklearnClassifier(n_jobs=4)
cls.fit(metaframe_train[train_columns], metaframe_train[test_columns], feat_type=['Categorical'] + ['Numerical'] * (len(train_columns) - 1), metric=f1_micro)

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

[WARNING] [2019-05-22 12:20:33,315:EnsembleBuilder(1791095845):39af627e4c2f33e5c297eec0322fbf1c] No models better than random - using Dummy Score!
[WARNING] [2019-05-22 12:20:33,363:EnsembleBuilder(1791095845):39af627e4c2f33e5c297eec0322fbf1c] No models better than random - using Dummy Score!


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv

[WARNING] [2019-05-22 12:41:27,157:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-05-22 12:41:27,157:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


...

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

[WARNING] [2019-05-22 13:10:17,259:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-05-22 13:10:17,259:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_f

AutoSklearnClassifier(delete_output_folder_after_terminate=False,
           delete_tmp_folder_after_terminate=False,
           disable_evaluator_output=False, ensemble_memory_limit=1024,
           ensemble_nbest=50, ensemble_size=50, exclude_estimators=None,
           exclude_preprocessors=None, get_smac_object_callback=None,
           include_estimators=None, include_preprocessors=None,
           initial_configurations_via_metalearning=25, logging_config=None,
           metadata_directory=None, ml_memory_limit=3072, n_jobs=4,
           output_folder=None, per_run_time_limit=360,
           resampling_strategy='holdout',
           resampling_strategy_arguments=None, seed=1, shared_mode=False,
           smac_scenario_args=None, time_left_for_this_task=3600,
           tmp_folder=None)

In [9]:
predicted = cls.predict(metaframes[-1][train_columns], n_jobs=4)

In [15]:
confusion_matrix(metaframes[-1][test_columns].values.flatten(), predicted.flatten())

array([[1924,  280],
       [ 390,  506]])

In [44]:
num_answers = len(test_columns) * len(metaframes[-1])
random_sample = np.zeros(num_answers, dtype='int')
random_sample[np.random.choice(num_answers, replace=False, size=metaframes[-1][test_columns].values.sum())] = 1
random_sample = random_sample.reshape(metaframes[-1][test_columns].values.shape)
f1_micro(metaframes[-1][test_columns], random_sample)

0.2857142857142857

In [18]:
metaframes_numbered = []
c_index = 0
for frame in metaframes:
    metaframes_numbered.append(frame.set_index(np.arange(c_index, c_index + len(frame))))
    c_index += len(frame)
metaframe = pd.concat(metaframes_numbered)
metaframe

,index,dataname,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,arsenic_male_bladder,9.126704,2.321928,4,5.578625,1.000000,9.600139,104.073434,0.060484,...,0,1,0,1,1,0,1,0,0,0
1,1,breast_tissue,6.714246,3.321928,1,3.953497,0.293765,0.015479,-1.196633,0.225433,...,0,0,0,0,0,1,0,0,0,0
2,1,fri_c3_1000_10,9.965784,3.459432,1,6.086775,0.292593,0.228655,-0.705256,0.140777,...,0,0,0,1,1,0,1,0,0,0
3,1,fri_c4_500_25,8.965784,4.700440,0,0.000000,0.126482,0.107719,-0.941671,0.319531,...,0,0,0,0,1,1,1,0,0,0
4,1,heart_h,8.194757,3.807355,13,4.836237,1.000000,-0.455030,2.491825,0.157464,...,0,0,0,0,0,0,0,0,0,0
5,1,prnn_fglass,7.734710,3.321928,3,3.960479,0.458348,1.384101,10.303645,0.148784,...,1,1,1,1,1,0,0,0,0,0
6,1,seismic_bumps,7.714246,3.000000,1,4.885009,0.708648,0.265505,-0.742847,0.052788,...,0,0,0,0,0,0,0,0,1,0
7,1,volcanoes_a4,10.565102,2.000000,1,4.913694,0.519282,0.506737,-0.146732,0.029444,...,0,0,0,0,1,1,1,1,1,0
8,1,wholesale_customers,8.781360,3.169925,3,5.568911,0.499404,5.131418,49.669519,0.120169,...,0,0,0,0,1,1,1,1,0,0
9,2,arsenic_male_bladder,9.126704,2.321928,4,5.578625,1.000000,9.600139,104.073434,0.060484,...,0,1,0,1,1,0,0,1,0,0


In [19]:
metaframe.to_csv('metaframe.csv')

In [52]:
datanames

NameError: name 'datanames' is not defined